In [50]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

class Node:
    question_number = -1
    follow_up_number = 0
    raw_question = ""
    shortened_question = ""
    references = []
    instructor_answer = ""
    student_answer = ""
    question_folder = ""
    
    
    def __init__(self, question_number, follow_up_number, raw_question, shortened_question, instructor_answer, student_answer, question_folder):
        self.question_number = question_number
        self.raw_question = raw_question
        self.follow_up_number = follow_up_number
        self.raw_question = raw_question
        self.instructor_answer = instructor_answer
        self.student_answer = student_answer
        self.shortened_question = shortened_question
        self.question_folder = question_folder
        
    def __str__(self):
        return self.question_folder + " " + self.shortened_question
    
    def __repr__(self):
        return self.question_folder + " " + self.shortened_question
    
    def add_references(self, all_refs):
        self.references = all_refs

In [51]:
import nltk
from nltk.tokenize import word_tokenize
import string 

def shortenQuestions(stop_words, question):

    shortened_questions = []

    tokens = word_tokenize(question)
    tokens = [w.lower() for w in tokens]
    words = [w for w in tokens if (not w in stop_words and w != "" and w != " ")]

    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    words = [w.translate(table) for w in words]

    final_question = " ".join(words)
    return final_question

def createStopWords(questions):
    
    questionsText = " ".join(questions)
    
    stop_words = ["ourselves", "hers", "between", "yourself", "but", "again", 
                      "there", "about", "once", "during", "out", "very", "having", 
                      "with", "they", "own", "an", "be", "some", "for", "do", "its", 
                      "yours", "such", "into", "of", "most", "itself", "other", "off",
                      "is", "s", "am", "or", "who", "as", "from", "him", "each", "the",
                      "themselves", "until", "below", "are", "we", "these", "your", "his", 
                      "through", "don", "nor", "me", "were", "her", "more", "himself", "this",
                      "down", "should", "our", "their", "while", "above", "both", "up", "to",
                      "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them",
                      "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", 
                      "then", "that", "because", "what", "over", "why", "so", "can", "did", 
                      "not", "now", "under", "he", "you", "herself", "has", "just",
                      "where", "too", "only", "myself", "which", "those", "i", "after",
                      "few", "whom", "t", "being", "if", "theirs", "my", "against", "a",
                      "by", "doing", "it", "how", "further", "was", "here", "than", "ok",
                      "thank", "ah"]
        
    entireTextTokens = nltk.tokenize.word_tokenize(questionsText)
    
    # Get rid of punctuation
    table = str.maketrans('', '', string.punctuation)
    entireTextTokens = [w.translate(table) for w in entireTextTokens]

    entireTextDist = nltk.FreqDist(w.lower() for w in entireTextTokens if w and w not in stop_words)

    mostCommon = entireTextDist.most_common(30)

    for i in mostCommon:
        stop_words.append(i[0])
        print(i[0])
        
    return stop_words

In [52]:
import csv
import datetime

filename = 'contributions.csv'

# reading csv file 
with open(filename, 'r', encoding="utf8") as csvfile: 
    # creating a csv reader object 
    csvreader = csv.reader(csvfile) 
      
    # extracting field names through first row 
    fields = next(csvreader) 
    
    rows = []
    entireText = ""
    
    # extracting each data row one by one 
    for row in csvreader: 
        rows.append(row) 
        entireText += " ".join(row)
        
    stop_words = createStopWords(entireText)
    
    curr_text = ""
    curr_post_number = "";
    curr_folder = ""
    curr_timestamp = ""
    final_column_question = ""
    final_column_instructor_answer = ""
    final_column_student_answer = ""
    all_contributions = {}
    raw_references = {}
    
    for row in rows[:]: 
        # parsing each column of a row 
        for col,field in zip(row, fields): 
            if(field == "Post Number"):
                if not curr_post_number == "" :
                    shortened_question = shortenQuestions(stop_words, final_column_question)

                    node = Node(curr_post_number, 0, final_column_question, shortened_question, final_column_instructor_answer, final_column_student_answer, curr_folder)
                    all_contributions[(curr_post_number, 0)] = node
                    for loc in find_all(final_column_instructor_answer, "@"):
                        next_space = loc + 1
                        while(next_space <= len(final_column_instructor_answer)-1 and final_column_instructor_answer[next_space].isdigit()):
                            next_space = next_space + 1

                        question = -1
                        try:
                            question = int(final_column_instructor_answer[(loc + 1):next_space])
                        except ValueError:
                            question = -1
                        if curr_post_number in raw_references:
                            raw_references_q = raw_references[curr_post_number]
                            raw_references_q.append(question)
                            raw_references.update({curr_post_number : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(question)
                            raw_references.update({curr_post_number : raw_references_q})
                                
                        if question in raw_references:
                            raw_references_q = raw_references[question]
                            raw_references_q.append(curr_post_number)
                            raw_references.update({question : raw_references_q})
                        else:
                            raw_references_q = []
                            raw_references_q.append(curr_post_number)
                            raw_references.update({question : raw_references_q})
                curr_post_number = col

            elif(field == "Submission HTML Removed"): #the question could also be in subject
                curr_text = col
                
            elif(field == "Folders"):
                curr_folder = col
                
            elif(field == "Created At"):
                curr_timestamp = col
                
            elif(field == "Part of Post"):
                if(col == "started_off_question"):
                    final_column_question = curr_text

                elif(col == "updated_question"):
                    final_column_question = curr_text
                        
                elif(col == "updated_i_answer"):
                    final_column_instructor_answer = curr_text
                    
                elif(col == "started_off_i_answer"):
                    final_column_instructor_answer = curr_text
                    
                elif(col == "updated_s_answer"):
                    final_column_student_answer = curr_text

                elif(col == "started_off_s_answer"):
                    final_column_student_answer = curr_text
                    
                elif(col == "followup"):
                    continue
                    
                elif(col == "reply_to_followup"):
                    continue

# print(raw_references)

for key in raw_references:
    if not key == -1:
        all_contributions[(str(key),0)].add_references(raw_references[key])

for key in all_contributions:
    print(key)
    contribution = all_contributions[key]
    print("Question Number: %s \n Question: %s \n Shortened Question: %s \n Answer: %s \n %s \n" % \
          (contribution.question_number, contribution.raw_question, contribution.shortened_question, contribution.instructor_answer, contribution.references))

e
o
n
r
h
l
u
c
d
p
f
m
w
g
y
1
b
2
0
k
v
3
i
x
4
t
8
j
9
5
('6', 0)
Question Number: 6 
 Question:  
 Shortened Question:  
 Answer:  
 [] 

('7', 0)
Question Number: 7 
 Question: Hi! I was registered half an hour ago for both the lecture and recitation and just tried to switch my recitation to better fit my schedule. I dropped my old recitation and when I tried to add a new one the, the system wouldn't let me. I would like to switch into 214, 216, 217, all of which Penn in Touch says are open. I would, at this point, also be happy to take my old spot back, which was in 211, which I believe is now closed. I am very concerned that without the recitation slot, Penn in Touch will soon delete my lecture spot, as well. Could someone please help me re-register for recitation? 

Thank you!
Shira Silver
Penn ID: 76417234 
 Shortened Question: hi  registered half hour ago lecture recitation tried switch recitation better fit schedule  dropped old recitation tried add new one  system would nt 


('929', 0)
Question Number: 929 
 Question: For part 2 of 4B, why is the answer 'not useful'? Is it because the signature does not contain an empty and therefore it cannot produce something nontrivial? Is nontrivial an empty list in this case?

For part 3 of 4B, why is the answer 'not safe'? Is it because the add function is defined for an integer? 
 Shortened Question: part 4b  answer not useful   signature contain empty therefore produce something nontrivial  nontrivial empty list case  part 4b  answer not safe   add function defined integer  
 Answer: For part 2, yes, you cannot actually ever create a priority queue.

For part 3, because we can call add directly on a list, we could break the internal invariants by providing a list that does not satisfy the priority queue invariants. 
 ['933', '934'] 

('930', 0)
Question Number: 930 
 Question: For functions such as delete, truncate, and enq, they all result in a unit.

For example 

val truncate : 'a -&gt; 'a queue -&gt; unit
What


('1751', 0)
Question Number: 1751 
 Question: I don't understand which part of my code is making my checkbox is at the top and missing like that. I don't know where I should fix this. 

 
 Shortened Question: nt understand part code making checkbox top missing like  nt know fix  
 Answer: Your implementation right now will only change the checkbox if the box itself is checked. You should handle clicks to the label as well. 
 [1380] 

('1752', 0)
Question Number: 1752 
 Question:  
 Shortened Question:  
 Answer: Yes 
 [] 

('1753', 0)
Question Number: 1753 
 Question: I'm having trouble with keeping the slider widget filled in once I have slid it a certain amount. When I'm sliding it, the widget gets filled in with a black rectangle, but as soon as I let go, it becomes white again. I have this functionality in my handle function for the widget right now, and I'm not sure if that's the best place to put it? 
 Shortened Question: m trouble keeping slider widget filled slid certain amoun

('2603', 0)
Question Number: 2603 
 Question: For both Java and OCaml ASM,

If say you have some object o which is not a primitive type,
let x = o;Object x = o;
if o is is in the heap, then x is on the stack
Then what happens to o when you set x to a different object p? Does o completely disappear from the stack or is it left there with nothing referenced to it?
let x = p;Object x = p; 
 Shortened Question: java ocaml asm  say object primitive type  let   object   heap  stack happens set different object  completely disappear stack left nothing referenced  let   object   
 Answer: Walking through this, you first update the pointer of the stack. x (a binding on the stack) used to point to o, but after the second line is run, x now points to p. 

o was never on the stack so it doesn't "disappear" from the stack and still exists in the heap until usually garbage collection (removal of memory that can no longer be accessed from the stack) occurs. If you take later CIS courses (240, 380 esp

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [53]:
# Create a map of shortened questions to nodes so we can cluster and then refer back
# For now, limit this to just one hw assignment

folder = "hw4"

to_cluster = {}
for node in all_contributions.values():
    if node.question_folder == folder:
        to_cluster[node.shortened_question] = node

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from xml.dom import minidom
keys = list(to_cluster.keys())

print(len(keys))

# parse an xml file by name
mydoc = minidom.parse('Posts.xml')

SEitems = mydoc.getElementsByTagName('row')

SEitems_questions = []
for item in SEitems:
    SEitems_questions.append(item.attributes['Body'].value)
    
SEitems_questions = SEitems_questions[1:6000]

vectorizer = TfidfVectorizer(stop_words='english')
X_orig = vectorizer.fit(SEitems_questions)
X = vectorizer.transform(keys)

# Number of clusters per homework
true_k = 50
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=10)
model.fit(X)

# print("\n")
# print("Prediction")




369


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=50, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [58]:
Y = vectorizer.transform(["Should we fail for empty queues for delete and contains too?"])
prediction = model.predict(Y)

In [60]:
final_clusters = {}
cluster_location = {}
for i in range(len(keys)):
    label = model.labels_[i]
    final_clusters.setdefault(label, []).append(to_cluster[keys[i]])
    cluster_location[to_cluster[keys[i]]] = label
    
# print(raw_references)
    
# print(final_clusters)

In [84]:
from unionfind import unionfind

length = len(list(final_clusters.keys()))
u = unionfind(length)

# Hard coded for not follow ups right now
# Getting some errors -- I think with questions that are outside this hw assignment
for key in cluster_location.keys():
    key_cluster_number = cluster_location[key]
    print(key.question_number)
    print(key.references)
    for ref in key.references:
        if not ref == -1:
            try:
                num_to_unite = cluster_location[all_contributions[(str(ref), 0)]]
                u.unite(key_cluster_number, num_to_unite)
            except:
                print("err")

print(u.groups())

789
[801]
801
['789']
802
[]
803
['903', '904', '905', '905', '905', '905', '905', '905', '906', '935', '936', '936', '936', '936']
err
err
err
err
err
806
['916', '917', '1064', '1065', '1194', '1194', '1194', '1195', '1209', '1209', '1210', '1215', '1216', '1219', '1220']
err
err
err
err
808
[554]
err
809
[]
810
[]
811
[777]
err
812
[777]
err
813
['895', '895', '895', '896', '896', '896']
814
['835', '836', '990', '990', '990', '991']
815
[]
816
[]
817
[]
818
[]
820
[]
1331
[]
822
[]
823
[]
824
[]
825
[]
830
[826]
err
831
[]
832
[]
833
['870', '871']
834
[]
835
[814]
836
[814]
837
[]
838
[]
840
[]
841
[]
844
[]
845
[]
846
[]
847
[]
848
['1063', '1064', '1064']
849
[]
850
[]
851
[]
853
['914', '915']
err
854
[]
855
['912', '926', '946', '947', '947', '948', '993', '994', '996', '997']
856
[]
857
[]
858
[]
859
[]
860
['880', '881']
err
862
[]
863
[]
864
['1249', '1249', '1249', '1249', '1249', '1249', '1249', '1249', '1249', '1249', '1249', '1250']
err
865
[777]
err
866
[777]
err
867
[